#Preliminary Analysis

In [ ]:
#Import statements (again to make sure they are run before running the code below)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
#Read in data and drop useless features (index has no meaning, the state is VA for all)
poisoning = pd.read_csv('poisoning_full_data.csv')
poisoning = poisoning.drop('index', axis=1)
poisoning = poisoning.drop('State', axis=1)

##Describe & Visualize Data

In [ ]:
num_list = list(poisoning.describe().columns) #get numerical features
cat_list = list(set(poisoning.columns) - set(poisoning.describe().columns)) #get categorical features

In [ ]:
#Visualize poisoning death categories against each numerical variable (everything but county/region)
#there is no apparent correlation
death_order = ['2-3.9', '4-5.9', '6-7.9','8-9.9','10-11.9', '12-13.9', '14-15.9', '16-17.9', '18-19.9', '20-21.9', '22-23.9', '24-25.9', '26-27.9', '28-29.9', '30+']

In [ ]:
#We notice that most features here have some population dependence in the data, so let's take a look at these by engineering some additional features
poisoning["labor_force_per_pop"] = poisoning['LaborForce']/poisoning['Population']
poisoning["employment_per_pop"] = poisoning['Employment']/poisoning['Population']
poisoning["unemployment_per_pop"] = poisoning['Unemployment']/poisoning['Population']
poisoning["hispanic_per_pop"] = poisoning['Total Hispanic/Latino']/poisoning['Population']
poisoning["white_per_pop"] = poisoning['Total White']/poisoning['Population']
poisoning["black_per_pop"] = poisoning['Total Black or African American']/poisoning['Population']
poisoning["native_per_pop"] = poisoning['Total American Indian or Alaskan Native']/poisoning['Population']
poisoning["asian_per_pop"] = poisoning['Total Asian']/poisoning['Population']
poisoning["pacific_per_pop"] = poisoning['Total Hawaiian / Pacific Islander']/poisoning['Population']
poisoning["other_per_pop"] = poisoning['Total other race']/poisoning['Population']
poisoning["nursery_enroll_per_pop"] = poisoning['Preschool/Nursery School Enrollment']/poisoning['Population']
poisoning["kindergarten_enroll_per_pop"] = poisoning['Kindegarten Enrollment']/poisoning['Population']
poisoning["G1-4_enroll_per_pop"] = poisoning['1st-4th grade enrollment']/poisoning['Population']
poisoning["drugBuys_per_pop"] = poisoning['NumDrugBuys']/poisoning['Population']
poisoning["medicare_enroll_per_pop"] = poisoning['Medicare Enrollment']/poisoning['Population']

engr_features = ["labor_force_per_pop","employment_per_pop","unemployment_per_pop","hispanic_per_pop","white_per_pop","black_per_pop","native_per_pop",
                 "asian_per_pop","pacific_per_pop","other_per_pop","nursery_enroll_per_pop","kindergarten_enroll_per_pop","G1-4_enroll_per_pop","drugBuys_per_pop","medicare_enroll_per_pop"]


##Preliminary Experiment: Random Forest

Data Cleaning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

X = poisoning.drop(['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)','UnemploymentRate',
 'labor_force_per_pop',
 'employment_per_pop',
 'unemployment_per_pop','nursery_enroll_per_pop',
 'kindergarten_enroll_per_pop',
 'G1-4_enroll_per_pop'], axis=1)
X = X.drop('Name', axis=1)
y = poisoning['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'].copy()

death_num = [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31]
death2num = dict(zip(death_order,death_num))
death_collapse1 = ['2-5.9','2-5.9','6-9.9','6-9.9','10-13.9','10-13.9', '14-17.9','14-17.9','18-21.9','18-21.9','22-25.9','22-25.9','26+','26+','26+']
deathReduce = dict(zip(death_order,death_collapse1))

death_collapse2 = ['2-7.9','2-7.9','2-7.9','8-13.9','8-13.9','8-13.9', '14-19.9','14-19.9','14-19.9','20-25.9','20-25.9','20-25.9','26+','26+','26+']
deathReduce2 = dict(zip(death_order,death_collapse2))

y_ord = []
y_red = []
y_red2 = []
for p in y:
  y_ord.append(death2num[p])
  y_red.append(deathReduce[p])
  y_red2.append(deathReduce[p])

num_list = list(X.describe().columns) #get numerical features
cat_list = list(set(X.columns) - set(poisoning.describe().columns)) #get categorical features

X_train, X_test, y_train, y_test = train_test_split(X, y_red, test_size=.2, random_state=42, stratify=y)

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()), #put all numerical data on same scale
    ])

num_attribs = num_list  #numerical values to fill in + scale values for
cat_attribs = cat_list #categorical values 

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs) ,
        ("cat", OneHotEncoder(), cat_attribs)],
    )

X_train = full_pipeline.fit_transform(X_train)
X_test = full_pipeline.transform(X_test)

{'2-3.9': '2-5.9', '4-5.9': '2-5.9', '6-7.9': '6-9.9', '8-9.9': '6-9.9', '10-11.9': '10-13.9', '12-13.9': '10-13.9', '14-15.9': '14-17.9', '16-17.9': '14-17.9', '18-19.9': '18-21.9', '20-21.9': '18-21.9', '22-23.9': '22-25.9', '24-25.9': '22-25.9', '26-27.9': '26+', '28-29.9': '26+', '30+': '26+'}


In [ ]:
#Try to do a neural network!!! 
from functools import partial

import tensorflow as tf
from tensorflow import keras

def softmax(x):
  return tf.nn.softmax(x+1e-150)

myModel = keras.models.Sequential([
    keras.layers.Dense(units = 256, input_shape=(34,), activation='relu' ),
    keras.layers.Dense(units=512, activation='relu'),
    keras.layers.Dense(units=1024, activation='relu'),
    keras.layers.Dense(units=2048, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(units=7, activation=softmax),
])

def classStringToUUID(s):
  return int(death_collapse1.index(s)/2)

y_train2 = [classStringToUUID(i) for i in y_train]

y_test2 = [classStringToUUID(i) for i in y_test]

X_train_final = np.array(X_train)
y_train_final = np.array(y_train2)

X_test_final = np.array(X_test)
y_test_final = np.array(y_test2)

In [ ]:
myEpochs = 2000
myOptimizer = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.01)
myLoss = "sparse_categorical_crossentropy"
myMetrics = ["accuracy"]
myModel.compile(loss= myLoss, optimizer = myOptimizer, metrics = myMetrics)
history = myModel.fit(x = X_train_final, y = y_train_final,
                      validation_data = (X_test_final, y_test_final), 
                      epochs = myEpochs)

Streaming output truncated to the last 5000 lines.
24/24 [==============================] - 0s 5ms/step - loss: 0.0306 - accuracy: 0.9906 - val_loss: 1.9399 - val_accuracy: 0.7914
Epoch 380/5000
24/24 [==============================] - 0s 5ms/step - loss: 0.0268 - accuracy: 0.9919 - val_loss: 1.9521 - val_accuracy: 0.7914
Epoch 381/5000
24/24 [==============================] - 0s 6ms/step - loss: 0.0408 - accuracy: 0.9866 - val_loss: 1.9321 - val_accuracy: 0.7968
Epoch 382/5000
24/24 [==============================] - 0s 5ms/step - loss: 0.0366 - accuracy: 0.9892 - val_loss: 1.9421 - val_accuracy: 0.7968
Epoch 383/5000
24/24 [==============================] - 0s 5ms/step - loss: 0.0259 - accuracy: 0.9946 - val_loss: 1.9481 - val_accuracy: 0.7968
Epoch 384/5000
24/24 [==============================] - 0s 5ms/step - loss: 0.0252 - accuracy: 0.9919 - val_loss: 1.9702 - val_accuracy: 0.7861
Epoch 385/5000
24/24 [==============================] - 0s 5ms/step - loss: 0.0289 - accuracy: 0.989

KeyboardInterrupt: ignored

In [ ]:
  a = myModel.predict(X_test_final, verbose=0)
  a = np.argmax(a, axis=1)
  a = tf.convert_to_tensor(a)
  confusion = tf.math.confusion_matrix(labels=y_test_final, predictions=a, num_classes=7, dtype='float32')

In [ ]:
print(confusion)

tf.Tensor(
[[ 6.  1.  0.  0.  0.  0.  0.]
 [ 1. 46.  4.  0.  0.  0.  0.]
 [ 0. 11. 46.  3.  0.  0.  0.]
 [ 0.  0.  9. 23.  1.  0.  0.]
 [ 0.  0.  0.  6. 10.  1.  0.]
 [ 0.  0.  0.  0.  1.  6.  0.]
 [ 0.  0.  0.  0.  0.  1. 11.]], shape=(7, 7), dtype=float32)


Recall class A: 6/7

Recall class B: 46/51

Recall class C: 46/60

Recall class D: 23/33

Recall class E: 10/17

Recall Class F: 6/7

Recall Class G: 11/12

Precision class A: 6/7

Precision class B: 46/58

Precision class C: 46/55

Precision class D: 23/32

Precision Class E: 10/12

Precision Class F: 6/8

Precision Class G: 1

f1 = 2*(Precision) (Recall) / (Precision + Recall)

f1 class A: 0.85714285714

f1 class B: 0.84403669724

f1 class C: .8

f1 class D: .70769230769

f1 class E: .68965517241

f1 class F: .8

f1 class G: 0.95652173913

Finally, taking the weighted averages of the f1 scores across all classes (weighted by how often the classes appeared in the test set):

Weighted average f1: 

(7xA + 58xB + 55xC + 32xD + 12xE + 8xF + 11xG)/(sum of all training values) = 0.80217422669



In [ ]:
myModel.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_151 (Dense)           (None, 256)               8960      
                                                                 
 dense_152 (Dense)           (None, 512)               131584    
                                                                 
 dense_153 (Dense)           (None, 1024)              525312    
                                                                 
 dense_154 (Dense)           (None, 2048)              2099200   
                                                                 
 dense_155 (Dense)           (None, 256)               524544    
                                                                 
 dropout_55 (Dropout)        (None, 256)               0         
                                                                 
 dense_156 (Dense)           (None, 7)               

In [ ]:
from scipy.stats.morestats import namedtuple
Charlottesville_City = poisoning[poisoning['Name']=="Charlottesville city"]
Fairfax_County = poisoning[poisoning['Name']=="Fairfax County"]
Albemarle = poisoning[poisoning['Name']=="Albemarle County"]
Montgomery = poisoning[poisoning['Name']=="Montgomery County"]
Richmond = poisoning[poisoning['Name']=="Richmond city"]

Cha = full_pipeline.transform(Charlottesville_City)
Fa = full_pipeline.transform(Fairfax_County)
Al = full_pipeline.transform(Albemarle)
Mo = full_pipeline.transform(Montgomery)
Ri = full_pipeline.transform(Richmond)

print("Threat level over the past 6 years: ")
print( "Charlottesville City:", np.argmax( myModel.predict(Cha, verbose=0), axis=1) )
print( "Fairfax County:", np.argmax( myModel.predict(Fa, verbose=0), axis=1) )
print( "Albemarle County:", np.argmax( myModel.predict(Al, verbose=0), axis=1) )
print( "Montgomery County (VTech):", np.argmax( myModel.predict(Mo, verbose=0), axis=1) )
print( "Richmond:", np.argmax( myModel.predict(Ri, verbose=0), axis=1) )


Threat level over the past 6 years: 
Charlottesville City: [2 1 2 1 2 2 2]
Fairfax County: [1 0 0 1 1 1 1]
Albemarle County: [2 1 0 0 0 0 0]
Montgomery County (VTech): [3 3 2 2 2 3 2]
Richmond: [4 5 4 4 5 3 5]


In [ ]:
max = [0, 0, 0, 0, 0]
county = ["", "", "", "" ,""]
for name in set(poisoning['Name']):
  County = poisoning[poisoning['Name']==name]
  c = full_pipeline.transform(County)
  c_pred = np.argmax( myModel.predict(c, verbose=0), axis=1)
  avg = sum(list(c_pred))/len(list(c_pred))
  if avg>max[0]:
    max[4] = max[3]
    county[4] = county[3]
    max[3] = max[2]
    county[3] = county[2]
    max[2] = max[1]
    county[2] = county[1]
    max[1] = max[0]
    county[1] = county[0]
    max[0] = avg
    county[0] = name
  elif avg>max[1]:
    max[4] = max[3]
    county[4] = county[3]
    max[3] = max[2]
    county[3] = county[2]
    max[2] = max[1]
    county[2] = county[1]
    max[1] = avg
    county[1] = name
  elif avg>max[2]:
    max[4] = max[3]
    county[4] = county[3]
    max[3] = max[2]
    county[3] = county[2]
    max[2] = avg
    county[2] = name
  elif avg>max[3]:
    max[4] = max[3]
    county[4] = county[3]
    max[3] = avg
    county[3] = name
  elif avg>max[4]:
    max[4] = avg
    county[4] = name

In [ ]:
print(county)

['Buchanan County', 'Dickenson County', 'Wise County', 'Russell County', 'Pulaski County']
